# Trabajando con datos desbalanceados

Al trabajar en *data science* es normal encontrarse con bases de datos en las que un tipo de dato (en particular, uno que es de interes) aparece con mucha menos frecuencia que el resto de los datos, esto en común en particular cuando se intentan detectar anormalidades en un conjunto de datos. Estos tipo de datos a tener una distribución en la que la gran mayoría de los casos van a ser "honestos". En general hablamos de datos desbalanceados cuando al rededor de un 5% o menos de los datos son del tipo que estámos buscando. En estos casos modelos de *machine learning* suelen producir málos clasificadores al encontrarse con datos sitrubuidos de está manera, y para esto exiten métodos con los que trabajar.

## Un ejemplo de datos desbalanceados

Por lo tanto, para resumir, al intentar resolver desafíos comerciales específicos con conjuntos de datos desequilibrados, los clasificadores producidos por algoritmos estándar de aprendizaje automático pueden no dar resultados precisos. Además de las transacciones fraudulentas, otros ejemplos de un problema comercial común con un conjunto de datos desequilibrado son:

* Conjuntos de datos para identificar la rotación de clientes donde una gran mayoría de clientes continuará utilizando el servicio. En concreto, empresas de telecomunicaciones en las que la tasa de rotación sea inferior al 2%.
* Conjuntos de datos para identificar enfermedades raras en diagnósticos médicos, etc.
* Desastres naturales como terremotos

## Desafíos con las técnicas estándar de aprendizaje automático

Los métodos de evaluación de modelos convencionales no miden con precisión el rendimiento del modelo cuando se enfrentan a conjuntos de datos desequilibrados.

Los algoritmos de clasificador estándar como el árbol de decisiones y la regresión logística tienen un sesgo hacia las clases que tienen un número de instancias. Tienden a predecir solo los datos de la clase mayoritaria. Las características de la clase minoritaria se tratan como ruido y a menudo se ignoran. Por lo tanto, existe una alta probabilidad de clasificación errónea de la clase minoritaria en comparación con la clase mayoritaria.

La evaluación del rendimiento de un algoritmo de clasificación se mide mediante la Matriz de confusión, que contiene información sobre la clase real y la prevista.

| Realidad       | Precicción              | Description             |
|----------------|-------------------------|-------------------------|
|                | Clase positiva          | Clase Negativa          |
| Clase positiva | Verdadero Positivo (TP) | Falso Negativo (FN)     |
| Clase negativa | Falso Positivo (FP)     | Verdadero Negativo (TN) |

$$ Acuracy=\frac{TP+TN}{TP+TN+FP+FN} $$

Sin embargo, mientras se trabaja en un dominio desequilibrado, la precisión no es una medida adecuada para evaluar el rendimiento del modelo. Por ejemplo: un clasificador que alcanza una precisión del 98% con una tasa de eventos del 2% no es exacto si clasifica todas las instancias como la clase mayoritaria. Y elimina el 2% de observaciones de clases minoritarias como ruido.

## ¿Como manejamos datos desbalanceados?
### A nivel de datos: técnicas de remuestreo
Tratar con conjuntos de datos desequilibrados implica estrategias como mejorar los algoritmos de clasificación o equilibrar las clases en los datos de entrenamiento (preprocesamiento de datos) antes de proporcionar los datos como entrada al algoritmo de aprendizaje automático. Se prefiere la última técnica ya que tiene una aplicación más amplia.

El objetivo principal de equilibrar las clases es aumentar la frecuencia de la clase minoritaria o disminuir la frecuencia de la clase mayoritaria. Esto se hace para obtener aproximadamente el mismo número de instancias para ambas clases. Veamos algunas técnicas de remuestreo:

Antes de esto, presetaremos los datos que usaremos, estas la base de datos de kaggle [*Porto Seguro’s Safe Driver Prediction*](https://www.kaggle.com/c/porto-seguro-safe-driver-prediction/data?select=train.csv), que es una base de datos en la que buscamos predecir si alguien usará o no su segturo de auto. Más datos en el link

In [1]:
import pandas as pd
import numpy as np

In [2]:
df = pd.read_csv('./input/train.csv', index_col = 'id').sample(10000)
df.columns = df.columns.str.strip().str.lower().str.replace(' ','_').str.replace('-','_')
df.head()

,target,ps_ind_01,ps_ind_02_cat,ps_ind_03,ps_ind_04_cat,ps_ind_05_cat,ps_ind_06_bin,ps_ind_07_bin,ps_ind_08_bin,ps_ind_09_bin,...,ps_calc_11,ps_calc_12,ps_calc_13,ps_calc_14,ps_calc_15_bin,ps_calc_16_bin,ps_calc_17_bin,ps_calc_18_bin,ps_calc_19_bin,ps_calc_20_bin
id,,,,,,,,,,,,,,,,,,,,,
131518,0,2,4,9,0,0,0,1,0,0,...,4,3,1,6,0,1,1,1,0,0
1224804,0,1,1,2,0,0,0,1,0,0,...,6,1,5,10,0,1,0,1,1,0
875922,1,5,1,6,0,2,0,0,0,1,...,6,2,1,4,0,0,1,0,1,0
210458,0,0,1,6,1,6,0,1,0,0,...,4,5,4,3,1,0,1,0,0,0
444536,0,2,1,1,0,0,1,0,0,0,...,4,0,1,9,0,1,1,0,0,1


In [3]:
df.groupby('target').apply(lambda x : x.target.count())

target
0    9633
1     367
dtype: int64

In [4]:
df.target.count()

10000

Podemos ver que tenemos alredeos de un 3% de datos en los que se hace uso del seguro. Y ahora veremos 3 de tecnicas con en fin de tratar de balancear los datos. Pero antes vamos a separar los datos con tal de dejar algunos para realizar una prueba despues.

In [5]:
from sklearn.model_selection import train_test_split 
X = df[df.columns[1:]]
y = df.target
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=1)
y_train.count(), y_train.value_counts()

(6700,
 0    6444
 1     256
 Name: target, dtype: int64)

Otro detalle, durante esta ayudantía usaremos los modulos `imblearn`, cuya documentación pueden encontrar [aquí](https://imbalanced-learn.readthedocs.io/), les ruego instalarlos en sus ambientes

#### Submuestreo aleatorio
El submuestreo aleatorio tiene como objetivo equilibrar la distribución de clases eliminando aleatoriamente los ejemplos de clases mayoritarias. Esto se hace hasta que se equilibren las instancias de las clases mayoritaria y minoritaria.

* Observaciones totales = 6700

* Observaciones fraudulentas = 6441

* Observaciones no fraudulentas = 259

* Tasa de eventos = 3.5%

En este caso, estamos tomando muestras del 10% sin reemplazo de instancias sin fraude. Y combinándolos con casos de fraude.

* Observaciones no fraudulentas después de un muestreo aleatorio = 259

* Observaciones totales después de combinarlas con observaciones fraudulentas = 259

* Tasa de eventos para el nuevo conjunto de datos después de un muestreo insuficiente = 25%

 

**Ventajas**
* Puede ayudar a mejorar el tiempo de ejecución y los problemas de almacenamiento al reducir la cantidad de muestras de datos de entrenamiento cuando el conjunto de datos de entrenamiento es enorme.
**Desventajas**
* Puede descartar información potencialmente útil que podría ser importante para crear clasificadores de reglas.
* La muestra elegida al azar bajo muestreo puede ser una muestra sesgada. Y no será un representante preciso de la población. Por lo tanto, resulta en resultados inexactos con el conjunto de datos de prueba real.

In [6]:
from imblearn.under_sampling import RandomUnderSampler
rus = RandomUnderSampler(random_state=42,  sampling_strategy = 1)
X_resampled_rus, y_resampled_rus = rus.fit_resample(X_train, y_train)
y_resampled_rus.value_counts() 

1    256
0    256
Name: target, dtype: int64

Notemos que usamos `sampling_strategy = 1` como el ratio de los datos que usaremos, i.e. $\frac{datos_{minoritrios}}{datos_{mayoritarios}}$, i.e., la misma cantidad de datos de los dos tipos.

#### Sobremuestreo aleatorio
El sobremuestreo aumenta el número de instancias en la clase minoritaria al replicarlas al azar para presentar una mayor representación de la clase minoritaria en la muestra.

* Observaciones totales = 6700

* Observaciones fraudulentas = 6441

* Observaciones no fraudulentas = 259

* Tasa de eventos = 3.5%

En este caso, estamos replicando 20 observaciones de fraude 20 veces.

* Observaciones no fraudulentas = 64548

* Observaciones fraudulentas después de replicar las observaciones de la clase minoritaria = 6441

* Observaciones totales en el nuevo conjunto de datos después del sobremuestreo = 6441

* Tasa de eventos para el nuevo conjunto de datos después de un muestreo insuficiente = 50%

**Ventajas**
* A diferencia del muestreo, este método no produce pérdida de información.
* Supera en rendimiento al submuestreo
**Desventajas**
* Aumenta la probabilidad de sobreajuste ya que replica los eventos de las clases minoritarias.

In [7]:
from imblearn.over_sampling import RandomOverSampler
ros = RandomOverSampler(random_state=0, sampling_strategy = 1)
X_resampled_ros, y_resampled_ros = ros.fit_resample(X_train, y_train)
y_resampled_ros.value_counts() 

1    6444
0    6444
Name: target, dtype: int64


#### Sobremuestreo informado: técnica sintética de sobremuestreo de minorías para datos desequilibrados (SMOTE) 
Se sigue esta técnica para evitar el sobreajuste que ocurre cuando se agregan réplicas exactas de instancias minoritarias al conjunto de datos principal. Se toma un subconjunto de datos de la clase minoritaria como ejemplo y luego se crean nuevas instancias sintéticas similares. Estas instancias sintéticas luego se agregan al conjunto de datos original. El nuevo conjunto de datos se utiliza como muestra para entrenar los modelos de clasificación.


* Observaciones totales = 6700

* Observaciones fraudulentas = 6441

* Observaciones no fraudulentas = 259

* Tasa de eventos = 3.5%

Se toma una muestra de 15 instancias de la clase minoritaria y se generan 20 veces instancias sintéticas similares

Después de la generación de instancias sintéticas, se crea el siguiente conjunto de datos

* Clase de minoría (observaciones fraudulentas) = 6441

* Clase mayoritaria (observaciones no fraudulentas) = 6441

* Tasa de eventos = 50%

 

**Ventajas**
* Mitiga el problema del sobreajuste causado por el sobremuestreo aleatorio, ya que se generan ejemplos sintéticos en lugar de la replicación de instancias.
* Sin pérdida de información útil
Desventajas
* Al generar ejemplos sintéticos, SMOTE no tiene en cuenta los ejemplos vecinos de otras clases. Esto puede resultar en un aumento en la superposición de clases y puede introducir ruido adicional
* SMOTE no es muy eficaz para datos de gran dimensión.


In [8]:
from imblearn.over_sampling import SMOTE
X_resampled_smote, y_resampled_smote = SMOTE(random_state = 42,sampling_strategy = 1).fit_resample(X_train, y_train)
y_resampled_smote.value_counts() 

1    6444
0    6444
Name: target, dtype: int64

Ahora, hagamos un test rápdio para ver como se comportan estos tres métodosk, para esto usaremos una *SVM* con cada uno de los :

In [9]:
from sklearn import svm
clf_ros = svm.SVC()
clf_rus = svm.SVC()
clf_smote = svm.SVC()
clf_ros.fit(X_resampled_ros, y_resampled_ros)
clf_rus.fit(X_resampled_rus, y_resampled_rus)
clf_smote.fit(X_resampled_smote, y_resampled_smote)
preds_ros = clf_ros.predict(X_test)
preds_rus = clf_rus.predict(X_test)
preds_smote = clf_smote.predict(X_test)

In [10]:
from sklearn.metrics import accuracy_score
acc_ros = accuracy_score(y_test, preds_ros)
acc_rus = accuracy_score(y_test, preds_rus)
acc_smote = accuracy_score(y_test, preds_smote)

print(f'Presicion Randon Over Sampling: {acc_ros}')
print(f'Presicion Random Under Sampling: {acc_rus}')
print(f'Presicion SMOTE: {acc_smote}')

Presicion Randon Over Sampling: 0.623030303030303
Presicion Random Under Sampling: 0.6827272727272727
Presicion SMOTE: 0.8009090909090909


Nuestros resultados del Over Sampling y el under Sampling son similares, pero SMOTE claramente muestra una mejora sobre los otros dos, 

Estás no son la únicas formas que existen de realizar este preoceso, de particular interes es *MSMOTE*, un algortimo que se basa en *SMOTE*, y *ADASYN*, un metodo totalmente diferente, ambos están bien explicados en la [documentacion](https://imbalanced-learn.readthedocs.io/en/stable/over_sampling.html#smote-variants) de *imblearn*, Pero además de estás existen muchas técnicas distintas que nos pueden ayudar a manejarnos con estos datos. 

## Actividad

Ahora usaremos otro *dataset* de kaggle, en este caso [Credit Card Fraud Detection](https://www.kaggle.com/mlg-ulb/creditcardfraud), en estos datos buscamos detectar que transacciones son fraudulentas, y cuales son reales. Para esto, tenemos 28 variables "incognitas" (solo sabemos que son el resultado de un algoritmo de reducción de dimensión), el monto de la transacción y si está es o no fraudulenta, veamos como es el dataset:

In [11]:
import pandas as pd
import numpy as np

In [12]:
credit_cards = pd.read_csv('./input/creditcard.csv').sample(10000) #si le tienen fe a su PC pueden aumentar este numero
credit_cards.columns = credit_cards.columns.str.strip().str.lower().str.replace(' ','_').str.replace('-','_')
credit_cards.head()

,time,v1,v2,v3,v4,v5,v6,v7,v8,v9,...,v21,v22,v23,v24,v25,v26,v27,v28,amount,class
145439,86945.0,-0.343694,0.720691,3.746195,5.094463,-1.545430,1.666376,-0.346986,0.264082,-0.064563,...,0.176104,1.149892,-0.085483,0.968228,-0.690584,0.447159,0.014067,-0.055585,82.74,0
69521,53461.0,-1.245406,0.573329,1.944124,-0.074705,-0.038978,2.131605,-0.706079,1.273053,-0.112872,...,0.232128,0.763191,0.008072,-0.980210,-0.612544,0.369466,-0.062142,0.017276,16.90,0
94935,65096.0,-0.351543,-0.300946,2.641319,-0.806061,-0.957076,0.691816,-0.465904,0.214132,-0.827099,...,-0.361567,-0.282835,-0.024364,0.035166,-0.607030,1.024715,-0.068631,-0.125628,22.44,0
139733,83321.0,1.199730,-0.599636,1.018696,0.634702,-1.186224,0.074857,-0.829334,0.037398,-0.422757,...,-0.379055,-0.581001,-0.029995,-0.150791,0.333131,-0.372862,0.084359,0.049202,64.00,0
59943,49080.0,-2.042650,0.767907,1.304693,1.140445,-2.345648,1.200739,-0.603841,1.197886,1.368857,...,-0.136099,0.145786,0.100748,0.117094,-0.410084,0.494700,0.079144,-0.010333,149.95,0


Veamos como el la distribución dentre las transacciónes fraudulentas y las que no:

In [14]:
credit_cards.groupby('class').apply(lambda x : x['class'].count())

class
0    9974
1      26
dtype: int64

Podemos ver que hay una mucha menos cantidad de transacciones fraudulentas que las que no lo son. Ahora, utiliza `SMOTE` y otro modelo que estimes convieniente (puede ser ROS, RUS, u otro de la documentación de *imblearn* que te llame la atención, recomiendo prestar atención a `ADASYN`, otro modelo ampliamente usado para esta finalidad), y luego compara su despeño en dos modelos, una SVM y otro de tu elección, el rendimiento de estos modelos. Tambien puedes jugar con el parámetro `sampling_strategy = x` y ver cual te genera menores resultados.